In [1]:
IMAGE_DIMS = (50,34,50,1)
OUTPUT_DIR = '../../../output/kaggle-bowl/step10/'
INPUT_DIR = '../../../input/kaggle-bowl/step5-50/'
DATASET_NAME_SUFFIX = '-centered-rotated'
LOAD_MODEL_FILE = None
#LOAD_MODEL_FILE = OUTPUT_DIR + 'tf-checkpoint-best7826'

In [2]:
import csv
import h5py
import numpy as np # linear algebra
import os
import logging
import tflearn

from modules.logging import logger
import modules.logging
import modules.lungprepare as lungprepare
import modules.utils as utils
import modules.cnn as cnn
from modules.utils import Timer

In [3]:
def start_training(model, input_dir, dataset_name_suffix, image_dims, output_dir):

    utils.mkdirs(output_dir, recreate=False)

    modules.logging.setup_file_logger(output_dir + 'out.log')
    
    dataset_path = utils.dataset_path(input_dir, 'test' + DATASET_NAME_SUFFIX, IMAGE_DIMS)
    with h5py.File(dataset_path, 'r') as train_hdf5:
        X = train_hdf5['X']
        Y = train_hdf5['Y']
        logger.info('X shape ' + str(X.shape))
        logger.info('Y shape ' + str(Y.shape))

        dataset_path = utils.dataset_path(input_dir, 'validate' + dataset_name_suffix, image_dims)
        with h5py.File(dataset_path, 'r') as validate_hdf5:
            X_validate = validate_hdf5['X']
            Y_validate = validate_hdf5['Y']
            logger.info('X_validate shape ' + str(X_validate.shape))
            logger.info('Y_validate shape ' + str(Y_validate.shape))

            logger.info('Starting CNN training...')
            model.fit(X, Y, validation_set=(X_validate, Y_validate), 
                      shuffle=True, batch_size=10, n_epoch=2,
                      show_metric=True, 
                      run_id='simplest1-'+str(image_dims))

In [6]:
logger.info('Prepare CNN for training')
network = cnn.net_simplest1(IMAGE_DIMS)
model = cnn.prepare_cnn_model(network, OUTPUT_DIR, model_file=None)

2017-03-07 11:53:57,046 INFO Prepare CNN for training
2017-03-07 11:53:57,106 INFO CNN model already loaded. Reusing it.


In [7]:
logger.info('Train CNN')
start_training(model, INPUT_DIR, DATASET_NAME_SUFFIX, IMAGE_DIMS, OUTPUT_DIR)
model.save(OUTPUT_DIR + 'final')
logger.info('==== ALL DONE ====')

2017-03-07 11:54:28,742 INFO Train CNN
2017-03-07 11:54:28,745 INFO X shape (764, 50, 34, 50, 1)
2017-03-07 11:54:28,747 INFO Y shape (764, 2)
2017-03-07 11:54:28,748 INFO X_validate shape (90, 50, 34, 50, 1)
2017-03-07 11:54:28,749 INFO Y_validate shape (90, 2)
2017-03-07 11:54:28,751 INFO Starting CNN training...


---------------------------------
Run id: simplest1-(50, 34, 50, 1)
Log directory: ../../../output/kaggle-bowl/step10/tf-logs/
---------------------------------
Training samples: 764
Validation samples: 90
--


KeyboardInterrupt: 

In [ ]:
logger.info('Evaluate model from dataset')
dataset_path = utils.dataset_path(INPUT_DIR, 'test', IMAGE_DIMS)
evaluate_dataset(dataset_path, model):